# Exercise 6

Klaus Hernando Rodríguez Cruz - Cod: 201124457

## SVM & Regularization


For this homework we consider a set of observations on a number of red and white wine varieties involving their chemical properties and ranking by tasters. Wine industry shows a recent growth spurt as social drinking is on the rise. The price of wine depends on a rather abstract concept of wine appreciation by wine tasters, opinion among whom may have a high degree of variability. Pricing of wine depends on such a volatile factor to some extent. Another key factor in wine certification and quality assessment is physicochemical tests which are laboratory-based and takes into account factors like acidity, pH level, presence of sugar and other chemical properties. For the wine market, it would be of interest if human quality of tasting can be related to the chemical properties of wine so that certification and quality assessment and assurance process is more controlled.

Two datasets are available of which one dataset is on red wine and have 1599 different varieties and the other is on white wine and have 4898 varieties. All wines are produced in a particular area of Portugal. Data are collected on 12 different properties of the wines one of which is Quality, based on sensory data, and the rest are on chemical properties of the wines including density, acidity, alcohol content etc. All chemical properties of wines are continuous variables. Quality is an ordinal variable with possible ranking from 1 (worst) to 10 (best). Each variety of wine is tasted by three independent tasters and the final rank assigned is the median rank given by the tasters.

A predictive model developed on this data is expected to provide guidance to vineyards regarding quality and price expected on their produce without heavy reliance on volatility of wine tasters.

In [32]:
import pandas as pd
import numpy as np

In [31]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [33]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
4725,6.9,0.310,0.33,12.7,0.038,33.0,116.0,0.99540,3.04,0.65,10.4,6,white
103,7.5,0.305,0.40,18.9,0.059,44.0,170.0,1.00000,2.99,0.46,9.0,5,white
3027,7.4,0.320,0.22,11.7,0.035,44.0,150.0,0.99578,3.10,0.45,10.4,5,white
2891,6.9,0.200,0.30,4.7,0.041,40.0,148.0,0.99320,3.16,0.35,10.2,6,white
4020,6.4,0.370,0.12,5.9,0.056,6.0,91.0,0.99536,3.06,0.46,8.4,4,white


# Exercise 6.1

Show the frecuency table of the quality by type of wine

In [34]:
func =lambda x:x.count()
pd.pivot_table(data, index='quality', columns='type', values= 'pH', aggfunc=func)

type,red,white
quality,,
3,10.0,20.0
4,53.0,163.0
5,681.0,1457.0
6,638.0,2198.0
7,199.0,880.0
8,18.0,175.0
9,NaN,5.0


# SVM

# Exercise 6.2

* Standarized the features (not the quality)
* Create a binary target for each type of wine
* Create two Linear SVM's for the white and red wines, repectively.


## Clasificación de base por tipo de vino (Rojo & Blanco)

## Estandarización de las características de los vinos

In [36]:
list(data.columns.values)

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality',
 'type']

In [37]:
data1= data[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide',
 'density','pH','sulphates','alcohol']]

In [38]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
sc.fit(data1)
data_std= sc.transform(data1)

In [43]:
data_std

array([[-0.16608919, -0.42318303,  0.28468605, ..., -1.35904886,
        -0.54617826, -1.41855821],
       [-0.70607349, -0.24094936,  0.14704613, ...,  0.50691489,
        -0.27735097, -0.83161516],
       [ 0.68245757, -0.36243847,  0.55996589, ...,  0.25811972,
        -0.61338508, -0.32852111],
       ...,
       [-0.70607349,  1.03468634, -1.29817304, ...,  1.25330039,
         1.47002637,  0.42611996],
       [-1.01463595,  1.85473786, -1.366993  , ...,  2.18628226,
         1.20119908, -0.2446721 ],
       [-0.93749534, -0.1802048 ,  1.04170561, ...,  1.06670401,
         0.86516498,  0.42611996]])

In [44]:
data1_std=pd.DataFrame(np.array(data_std),columns=['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide',
 'density','pH','sulphates','alcohol'])

In [46]:
data2=data[['quality','type']]

In [47]:
data_std=pd.concat([data1_std,data2],axis=1,sort=False)

In [49]:
datared = data_std[data_std['type']=='red']
datawhite = data_std[data_std['type']=='white']

## Creación del target por tipo de vino

In [51]:
# Taget para el vino blanco
datawhite['quality2']=np.where(datawhite.quality>5,'good','bad')
#Target para el vino rojo
datared['quality2']=np.where(datared.quality>5,'good','bad')

C:\Users\klaus_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\klaus_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


## SVM vino blanco

In [53]:
x_white= datawhite[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide',
 'density','pH','sulphates','alcohol']]
y_white= datawhite['quality2']

In [66]:
# Partción de la muesta en train (70%) y Test (30%) 
from sklearn.cross_validation import train_test_split
x_white_train,x_white_test,y_white_train, y_white_test= train_test_split(x_white,y_white,test_size=0.3,random_state=0)

In [67]:
from sklearn.svm import SVC
svm = SVC(kernel='linear', C=1.0, random_state=0)
svm.fit(x_white_train,y_white_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [83]:
y_white_pred=svm.predict(x_white_test)
#Test de Accuracy para el algoritmo SVM vino blanco
from sklearn.metrics import accuracy_score
print('Accuracy:%2f'% accuracy_score(y_white_test,y_white_pred))

Accuracy:0.735374


## SVM Vinos Rojos

In [84]:
x_red= datared[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide',
 'density','pH','sulphates','alcohol']]
y_red= datared['quality2']

In [85]:
# Partción de la muesta en train (70%) y Test (30%) 
from sklearn.cross_validation import train_test_split
x_red_train,x_red_test,y_red_train, y_red_test= train_test_split(x_red,y_red,test_size=0.3,random_state=0)

In [88]:
from sklearn.svm import SVC
svm_r = SVC(kernel='linear', C=1.0, random_state=0)
svm_r.fit(x_red_train,y_red_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [100]:
y_red_pred=svm_r.predict(x_red_test)
#Test de Accuracy para el algoritmo SVM vino rojo
from sklearn.metrics import accuracy_score
print('Accuracy:%2f'% accuracy_score(y_red_test,y_red_pred))

Accuracy:0.747917


# Exercise 6.3

Test the two SVM's using the different kernels (‘poly’, ‘rbf’, ‘sigmoid’)


## SVM's Vino Blanco

In [94]:
# Kernel 'poly'
svm.p_w = SVC(kernel='poly', C=1.0, random_state=0)
svm.p_w.fit(x_white_train,y_white_train)
y_white_pred_p=svm.p_w.predict(x_white_test)
#Test de Accuracy para el algoritmo SVM vino blanco
from sklearn.metrics import accuracy_score
print('Accuracy:%2f'% accuracy_score(y_white_test,y_white_pred_p))

Accuracy:0.723129


In [95]:
# Kernel 'rbf'
svm.b_w = SVC(kernel='rbf', C=1.0, random_state=0)
svm.b_w.fit(x_white_train,y_white_train)
y_white_pred_b=svm.b_w.predict(x_white_test)
#Test de Accuracy para el algoritmo SVM vino blanco
from sklearn.metrics import accuracy_score
print('Accuracy:%2f'% accuracy_score(y_white_test,y_white_pred_b))

Accuracy:0.768707


In [96]:
# Kernel 'sigmoid'
svm.s_w = SVC(kernel='sigmoid', C=1.0, random_state=0)
svm.s_w.fit(x_white_train,y_white_train)
y_white_pred_s=svm.s_w.predict(x_white_test)
#Test de Accuracy para el algoritmo SVM vino blanco
from sklearn.metrics import accuracy_score
print('Accuracy:%2f'% accuracy_score(y_white_test,y_white_pred_s))

Accuracy:0.666667


Para el caso del algoritmo de clasidicación para el grupo de vinos blancos, el algoritmo que proporciona un mayor accuracy es el algoritmo de kernel 'rbf'

## SVM's Vino Rojo

In [98]:
# Kernel 'poly'
svm.p_r = SVC(kernel='poly', C=1.0, random_state=0)
svm.p_r.fit(x_red_train,y_red_train)
y_red_pred_p=svm.p_r.predict(x_red_test)
#Test de Accuracy para el algoritmo SVM vino rojo
from sklearn.metrics import accuracy_score
print('Accuracy:%2f'% accuracy_score(y_red_test,y_red_pred_p))

Accuracy:0.735417


In [101]:
# Kernel 'rbf'
svm.b_r = SVC(kernel='rbf', C=1.0, random_state=0)
svm.b_r.fit(x_red_train,y_red_train)
y_red_pred_b=svm.b_r.predict(x_red_test)
#Test de Accuracy para el algoritmo SVM vino rojo
from sklearn.metrics import accuracy_score
print('Accuracy:%2f'% accuracy_score(y_red_test,y_red_pred_b))

Accuracy:0.731250


In [102]:
# Kernel 'sigmoid'
svm.s_r = SVC(kernel='sigmoid', C=1.0, random_state=0)
svm.s_r.fit(x_red_train,y_red_train)
y_red_pred_s=svm.s_r.predict(x_red_test)
#Test de Accuracy para el algoritmo SVM vino rojo
from sklearn.metrics import accuracy_score
print('Accuracy:%2f'% accuracy_score(y_red_test,y_red_pred_s))

Accuracy:0.618750


Para el caso del algoritmo de clasidicación para el grupo de vinos rojos, el algoritmo que proporciona un mayor accuracy es el algoritmo de kernel 'linear'

# Exercise 6.4
Using the best SVM find the parameters that gives the best performance

'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]

## SVM Vino Blanco

In [121]:
def SVM_white(c,g):
    svm.b_w = SVC(kernel='rbf', gamma= g, C=c, random_state=0)
    svm.b_w.fit(x_white_train,y_white_train)
    y_white_pred_b=svm.b_w.predict(x_white_test)
    #Test de Accuracy para el algoritmo SVM vino blanco
    from sklearn.metrics import accuracy_score
    return print('C= %f, gamma=%f,Accuracy:%2f'% (c,g, accuracy_score(y_white_test,y_white_pred_b)))


In [123]:
SVM_white(0.1,0.01)
SVM_white(0.1,0.001)
SVM_white(0.1,0.0001)
SVM_white(1,0.01)
SVM_white(1,0.001)
SVM_white(1,0.0001)
SVM_white(10,0.01)
SVM_white(10,0.001)
SVM_white(10,0.0001)
SVM_white(100,0.01)
SVM_white(100,0.001)
SVM_white(100,0.0001)
SVM_white(1000,0.01)
SVM_white(1000,0.001)
SVM_white(1000,0.0001)


C= 0.100000, gamma=0.010000,Accuracy:0.655782
C= 0.100000, gamma=0.001000,Accuracy:0.641497
C= 0.100000, gamma=0.000100,Accuracy:0.641497
C= 1.000000, gamma=0.010000,Accuracy:0.734014
C= 1.000000, gamma=0.001000,Accuracy:0.657143
C= 1.000000, gamma=0.000100,Accuracy:0.641497
C= 10.000000, gamma=0.010000,Accuracy:0.751701
C= 10.000000, gamma=0.001000,Accuracy:0.725850
C= 10.000000, gamma=0.000100,Accuracy:0.657143
C= 100.000000, gamma=0.010000,Accuracy:0.759864
C= 100.000000, gamma=0.001000,Accuracy:0.744898
C= 100.000000, gamma=0.000100,Accuracy:0.723810
C= 1000.000000, gamma=0.010000,Accuracy:0.763946
C= 1000.000000, gamma=0.001000,Accuracy:0.754422
C= 1000.000000, gamma=0.000100,Accuracy:0.732653


In [124]:
SVM_white(1,0.09)

C= 1.000000, gamma=0.090000,Accuracy:0.768027


Para el caso del modelo de clasificación del vino blanco, el mejor accuracy se alcanza con el algoritmo SVM 'rbf'con los parámetros C=1 y gamma=0.09

## SVM Vino Rojo

In [125]:
def SVM_red(c,g):
    svm_r = SVC(kernel='linear', gamma= g, C=c, random_state=0)
    svm_r.fit(x_red_train,y_red_train)
    y_red_pred=svm_r.predict(x_red_test)
    #Test de Accuracy para el algoritmo SVM vino blanco
    from sklearn.metrics import accuracy_score
    return print('C= %f, gamma=%f,Accuracy:%2f'% (c,g, accuracy_score(y_red_test,y_red_pred)))

In [130]:
SVM_red(0.1,0.01)
SVM_red(0.1,0.001)
SVM_red(0.1,0.0001)
SVM_red(1,0.01)
SVM_red(1,0.001)
SVM_red(1,0.0001)
SVM_red(10,0.01)
SVM_red(10,0.001)
SVM_red(10,0.0001)
SVM_red(100,0.01)
SVM_red(100,0.001)
SVM_red(100,0.0001)
SVM_red(1000,0.01)
SVM_red(1000,0.001)
SVM_red(1000,0.0001)

C= 0.100000, gamma=0.010000,Accuracy:0.752083
C= 0.100000, gamma=0.001000,Accuracy:0.752083
C= 0.100000, gamma=0.000100,Accuracy:0.752083
C= 1.000000, gamma=0.010000,Accuracy:0.747917
C= 1.000000, gamma=0.001000,Accuracy:0.747917
C= 1.000000, gamma=0.000100,Accuracy:0.747917
C= 10.000000, gamma=0.010000,Accuracy:0.750000
C= 10.000000, gamma=0.001000,Accuracy:0.750000
C= 10.000000, gamma=0.000100,Accuracy:0.750000
C= 100.000000, gamma=0.010000,Accuracy:0.750000
C= 100.000000, gamma=0.001000,Accuracy:0.750000
C= 100.000000, gamma=0.000100,Accuracy:0.750000
C= 1000.000000, gamma=0.010000,Accuracy:0.747917
C= 1000.000000, gamma=0.001000,Accuracy:0.747917
C= 1000.000000, gamma=0.000100,Accuracy:0.747917


Para el caso del modelo de clasificación del vino rojo, el mejor accuracy se alcanza con el algoritmo SVM 'linear'con los parámetros C=0.1 y gamma=0.01

# Exercise 6.5

Compare the results with other methods

## Regresión Logística - Vino Blanco

In [132]:
# Estimando un algoritmo de clasificación vía regresión Lineal
from sklearn.linear_model import LogisticRegression #Importando la librería de regresión Lineal con Sklearn
lr= LogisticRegression(solver='liblinear',C=1e9, random_state=0)# Definiendo la función "Lr" con sus parámetros
lr.fit(x_white_train,y_white_train) #Entrenando un algoritmo de Machine Learnig con el dataset x_train
y_pred =lr.predict_proba(x_white_test) # Predicción de los valores para X_test
import sklearn.metrics
score = lr.score(x_white_test, y_white_test) # Otra forma de medir el accuracy de un modelo de regresión
print('Accuracy:%2f'% score)

Accuracy:0.739456


Para el caso del modelo de clasificación del vino blanco, el mejor accuracy se alcanza con el algoritmo SVM 'rbf'con los parámetros C=1 y gamma=0.09

## Regresión Logística - Vino Rojo

In [133]:
# Estimando un algoritmo de clasificación vía regresión Lineal
from sklearn.linear_model import LogisticRegression #Importando la librería de regresión Lineal con Sklearn
lr= LogisticRegression(solver='liblinear',C=1e9, random_state=0)# Definiendo la función "Lr" con sus parámetros
lr.fit(x_red_train,y_red_train) #Entrenando un algoritmo de Machine Learnig con el dataset x_train
y_pred =lr.predict_proba(x_red_test) # Predicción de los valores para X_test
import sklearn.metrics
score = lr.score(x_red_test, y_red_test) # Otra forma de medir el accuracy de un modelo de regresión
print('Accuracy:%2f'% score)

Accuracy:0.747917


Para la clasificación de vino rojo, el mejor algoritmo sigue siendo SVM 'linear'con los parámetros C=0.1 y gamma=0.01

# Regularization

# Exercise 6.6


* Train a linear regression to predict wine quality (Continous)

* Analyze the coefficients

* Evaluate the RMSE

## Regresión Lineal para Vino- Blanco

In [197]:
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [198]:
x_white=data_w[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide',
 'density','pH','sulphates','alcohol']]
y_white=data_w['quality']

In [199]:
# Partción de la muesta en train (70%) y Test (30%) 
from sklearn.cross_validation import train_test_split
x_white_train,x_white_test,y_white_train, y_white_test= train_test_split(x_white,y_white,test_size=0.3,random_state=0)

In [200]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
sc.fit(x_white_train)
x_tr_white_std= sc.transform(x_white_train)

In [201]:
y_mean, y_std = y_white_train.mean(), y_white_train.std()
y_tr_white_std = (y_white_train - y_mean)/ y_std

In [202]:
# Estimación del modelo usando OLS
from sklearn.linear_model import LinearRegression
linreg = LinearRegression(fit_intercept=False)
linreg.fit(x_tr_white_std, y_tr_white_std)
linreg.coef_

array([ 0.10963305, -0.20719608, -0.00203301,  0.54931424, -0.01065429,
        0.09917651, -0.01159626, -0.6728308 ,  0.14996364,  0.08709927,
        0.18008521])

In [203]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
sc.fit(x_white_train)
x_tst_white_std= sc.transform(x_white_test)

In [204]:
y_tst_white_std = (y_white_test - y_mean)/ y_std

In [205]:
y_pred1 = linreg.predict(x_tst_white_std)
# Calculo MSE Metodología OLS
from sklearn import metrics
import numpy as np
print('MSE metodología OLS:', metrics.mean_squared_error(y_tst_white_std, y_pred1))

MSE metodología OLS: 0.786956309844809


In [217]:
# Estimación del modelo usando SDG
from sklearn.linear_model import SGDRegressor
linreg_SDG = SGDRegressor(fit_intercept=False, max_iter=500,tol = 0.0000001)
linreg_SDG.fit(x_tr_white_std, y_tr_white_std)
linreg_SDG.coef_

array([ 0.0054846 , -0.21362165,  0.00431453,  0.2769652 , -0.01165198,
        0.11299672, -0.02896232, -0.22637824,  0.05790943,  0.07240937,
        0.39374484])

In [218]:
y_pred2 = linreg_SDG.predict(x_tst_white_std)
# Calculo MSE Metodología OLS
from sklearn import metrics
import numpy as np
print('MSE metodología SDG:', metrics.mean_squared_error(y_tst_white_std, y_pred2))

MSE metodología SDG: 0.7804579897023612


## Regresión Lineal para Vino- Rojo

In [208]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')

In [209]:
x_red=data_r[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide',
 'density','pH','sulphates','alcohol']]
y_red=data_r['quality']

In [210]:
# Partción de la muesta en train (70%) y Test (30%) 
from sklearn.cross_validation import train_test_split
x_red_train,x_red_test,y_red_train, y_red_test= train_test_split(x_red,y_red,test_size=0.3,random_state=0)

In [211]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
sc.fit(x_red_train)
x_tr_red_std= sc.transform(x_red_train)
y_mean, y_std = y_red_train.mean(), y_red_train.std()
y_tr_red_std = (y_red_train - y_mean)/ y_std

In [212]:
# Estimación del modelo usando OLS
from sklearn.linear_model import LinearRegression
linreg = LinearRegression(fit_intercept=False)
linreg.fit(x_tr_red_std, y_tr_red_std)
linreg.coef_

array([ 0.04278913, -0.27324555, -0.02329091,  0.04009502, -0.11420775,
        0.02601282, -0.12493469, -0.03856832, -0.07488928,  0.18532489,
        0.34575834])

In [213]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
sc.fit(x_red_train)
x_tst_red_std= sc.transform(x_red_test)
y_tst_red_std = (y_red_test - y_mean)/ y_std

In [214]:
y_pred1 = linreg.predict(x_tst_red_std)
# Calculo MSE Metodología OLS
from sklearn import metrics
import numpy as np
print('MSE metodología OLS:', metrics.mean_squared_error(y_tst_red_std, y_pred1))

MSE metodología OLS: 0.5910660686504577


In [215]:
# Estimación del modelo usando SDG
from sklearn.linear_model import SGDRegressor
linreg_SDG = SGDRegressor(fit_intercept=False, max_iter=500,tol = 0.0000001)
linreg_SDG.fit(x_tr_red_std, y_tr_red_std)
linreg_SDG.coef_

array([ 0.07025671, -0.26376978, -0.01270404,  0.05992728, -0.12226253,
        0.01506832, -0.13533236, -0.0578797 , -0.06301727,  0.17203292,
        0.33446791])

In [216]:
y_pred2 = linreg_SDG.predict(x_tst_red_std)
# Calculo MSE Metodología OLS
from sklearn import metrics
import numpy as np
print('MSE metodología SDG:', metrics.mean_squared_error(y_tst_red_std, y_pred2))

MSE metodología SDG: 0.5958737769598367


# Exercise 6.7

* Estimate a ridge regression with alpha equals 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

## Ridge Regression- Vino Blanco

In [219]:
# alpha=0.1 
from sklearn.linear_model import Ridge
ridgereg1 = Ridge(alpha=0.1, normalize=True)
ridgereg1.fit(x_white_train, y_white_train)
y_pred1 = ridgereg1.predict(x_white_test)
print('MSE Ridge Regression alpha 0.1:',np.sqrt(metrics.mean_squared_error(y_white_test, y_pred1)))


MSE Ridge Regression alpha 0.1: 0.7797233024279019


In [220]:
# alpha=1 
from sklearn.linear_model import Ridge
ridgereg2 = Ridge(alpha=1, normalize=True)
ridgereg2.fit(x_white_train, y_white_train)
y_pred2 = ridgereg2.predict(x_white_test)
print('MSE Ridge Regression alpha 1:',np.sqrt(metrics.mean_squared_error(y_white_test, y_pred2)))

MSE Ridge Regression alpha 1: 0.8120347448335605


In [221]:
print(ridgereg1.coef_)

[-8.48885792e-03 -1.63467861e+00  9.50634401e-03  3.42636009e-02
 -1.75610555e+00  5.50671129e-03 -8.39582620e-04 -5.13078081e+01
  3.09478498e-01  4.17468570e-01  2.52495787e-01]


In [222]:
print(ridgereg2.coef_)

[-2.72539154e-02 -8.57988875e-01  5.04142556e-02  6.03593151e-03
 -2.45762035e+00  2.73257813e-03 -7.31960665e-04 -2.58883533e+01
  1.61366105e-01  2.21480222e-01  1.28845959e-01]


Comparando los coeficientes obtenidos para el modelo de regresión Lineal y el modelo de regresión Ridge se tienen que todos los coeficientes son menores para el caso de la regresión ridge debido a la restricción adicional incluida en el proceso de optimización.
Adicionalmente el accuracy del modelo de regresión ridge con alpha=1 es mayor que el accuracy obtenido con la estimación del modelo lineal vía OLS.

## Ridge Regression- Vino Rojo

In [223]:
# alpha=0.1 
from sklearn.linear_model import Ridge
ridgereg1 = Ridge(alpha=0.1, normalize=True)
ridgereg1.fit(x_red_train, y_red_train)
y_pred1 = ridgereg1.predict(x_red_test)
print('MSE Ridge Regression alpha 0.1:',np.sqrt(metrics.mean_squared_error(y_red_test, y_pred1)))

MSE Ridge Regression alpha 0.1: 0.6344194631893536


In [224]:
# alpha=1 
from sklearn.linear_model import Ridge
ridgereg2 = Ridge(alpha=1, normalize=True)
ridgereg2.fit(x_red_train, y_red_train)
y_pred2 = ridgereg2.predict(x_red_test)
print('MSE Ridge Regression alpha 0.1:',np.sqrt(metrics.mean_squared_error(y_red_test, y_pred2)))

MSE Ridge Regression alpha 0.1: 0.6569663486954709


In [225]:
print(ridgereg1.coef_)

[ 2.68276389e-02 -1.08428984e+00  5.65341014e-02  2.36543153e-02
 -1.76994118e+00  1.12334507e-03 -2.71388369e-03 -3.00009155e+01
 -2.54181268e-01  7.95554880e-01  2.33458671e-01]


In [226]:
print(ridgereg2.coef_)

[ 1.83712556e-02 -6.75208907e-01  2.49195784e-01  1.00770556e-02
 -1.01625393e+00 -8.36569442e-04 -1.63673406e-03 -2.68060154e+01
 -1.00168476e-01  4.58489960e-01  1.41011129e-01]


Comparando los coeficientes obtenidos para el modelo de regresión Lineal y el modelo de regresión Ridge se tienen que todos los coeficientes son menores para el caso de la regresión ridge debido a la restricción adicional incluida en el proceso de optimización.
Adicionalmente el accuracy del modelo de regresión ridge con alpha=1 es mayor que el accuracy obtenido con la estimación del modelo lineal vía OLS.

# Exercise 6.8

* Estimate a lasso regression with alpha equals 0.01, 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

## Lasso Regression

In [287]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [288]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)

In [274]:
x=data[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide',
 'density','pH','sulphates','alcohol']]
y=data['quality']

In [276]:
# Partición de la muesta en train (70%) y Test (30%) 
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train, y_test= train_test_split(x,y,test_size=0.3,random_state=0)

In [282]:
# Lasso Regression alpha=0.01
from sklearn.linear_model import Lasso
lassoreg1 = Lasso(alpha=0.001, normalize=True, max_iter=100000000)
lassoreg1.fit(x_train, y_train)
# calculate RMSE (for alpha=0.001)
y_pred1 = lassoreg1.predict(x_test)
print('MSE Lasso Regression alpha 0.001:',np.sqrt(metrics.mean_squared_error(y_test, y_pred1)))
print("Número de variables usadas:{}".format(np.sum(lassoreg1.coef_!=0)))

MSE Lasso Regression alpha 0.001: 0.7536931857261449
Número de variables usadas:3


In [270]:
# Lasso Regression alpha=0.1
from sklearn.linear_model import Lasso
lassoreg2 = Lasso(alpha=0.1, normalize=True, max_iter=100000000)
lassoreg2.fit(x_train, y_train)
# calculate RMSE (for alpha=0.01)
y_pred2 = lassoreg2.predict(x_test)
print('MSE Lasso Regression alpha 0.1:',np.sqrt(metrics.mean_squared_error(y_test, y_pred2)))
print("Número de variables usadas:{}".format(np.sum(lassoreg2.coef_!=0)))

MSE Lasso Regression alpha 0.1: 0.866069106529371
Número de variables usadas:0


In [271]:
# Lasso Regression alpha=1
from sklearn.linear_model import Lasso
lassoreg3 = Lasso(alpha=1, normalize=True, max_iter=100000000)
lassoreg3.fit(x_train, y_train)
# calculate RMSE (for alpha=1)
y_pred3 = lassoreg3.predict(x_test)
print('MSE Lasso Regression alpha 1:',np.sqrt(metrics.mean_squared_error(y_test, y_pred3)))
print("Número de variables usadas:{}".format(np.sum(lassoreg3.coef_!=0)))

MSE Lasso Regression alpha 1: 0.866069106529371
Número de variables usadas:0


In [254]:
 print(np.around(lassoreg1.coef_,5))

[ 6.920000e-03 -1.812410e+00 -0.000000e+00  5.132000e-02 -8.706700e-01
  4.910000e-03 -3.000000e-04 -8.206488e+01  3.893200e-01  4.415200e-01
  2.594200e-01]


In [241]:
 print(np.around(lassoreg3.coef_,5))

[-0. -0. -0. -0. -0.  0. -0. -0.  0.  0.  0.]


In [242]:
 print(np.around(lassoreg3.coef_,5))

[-0. -0. -0. -0. -0.  0. -0. -0.  0.  0.  0.]


# Exercise 6.9

* Create a binary target

* Train a logistic regression to predict wine quality (binary)

* Analyze the coefficients

* Evaluate the f1score

In [310]:
# Taget para el vino blanco
data['quality2']=np.where(data.quality>6,1,0)

In [311]:
x=data[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide',
 'density','pH','sulphates','alcohol']]
y= data['quality2']

In [312]:
# Partción de la muesta en train (70%) y Test (30%) 
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train, y_test= train_test_split(x,y,test_size=0.3,random_state=0)

In [313]:
#Estandarización de la base x
from sklearn.preprocessing import StandardScaler
sc= StandardScaler() #definición de la función de estandarización
sc.fit(x_train) # defino con que media y varianza se ajustara los datos
x_train_std = sc.transform(x_train) # Definición de la variable estandarizada
x_test_std = sc.transform(x_test)

In [314]:
from sklearn.linear_model import LogisticRegression #Importando la librería de regresión Lineal con Sklearn
lr= LogisticRegression(solver='liblinear',C=1e9, random_state=0)# Definiendo la función "Lr" con sus parámetros
lr.fit(x_train_std,y_train) #Entrenando un algoritmo de Machine Learnig con el dataset x_train_std

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=0,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [315]:
lr.coef_

array([[ 0.6045442, -0.5517844, -0.0600713,  0.8471515, -0.1801041,
         0.2545709, -0.339323 , -0.8737092,  0.3902161,  0.3530193,
         0.7605953]])

In [316]:
y_pred =lr.predict(x_test_std)
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(y_test, y_pred)

0.3668430335097002

# Exercise 6.10

* Estimate a regularized logistic regression using:
* C = 0.01, 0.1 & 1.0
* penalty = ['l1, 'l2']
* Compare the coefficients and the f1score

## Comparación f1

In [324]:
def lr_ridge(c):
    lr_ridge= LogisticRegression(solver='liblinear',C=c,penalty='l2',random_state=0)# Definiendo la función "Lr" con sus parámetros
    lr_ridge.fit(x_train_std,y_train) #Entrenando un algoritmo de Machine Learnig con el dataset x_train_std
    y_pred =lr_ridge.predict(x_test_std)
    from sklearn.metrics import f1_score
    return print('C= %f, penalty=L2,f1 test:%2f'% (c, sklearn.metrics.f1_score(y_test, y_pred)))

In [325]:
lr_ridge(0.01)
lr_ridge(0.1)
lr_ridge(1)

C= 0.010000, penalty=L2,f1 test:0.346225
C= 0.100000, penalty=L2,f1 test:0.363636
C= 1.000000, penalty=L2,f1 test:0.366843


In [326]:
def lr_lasso(c):
    lr_lasso= LogisticRegression(solver='liblinear',C=c,penalty='l1',random_state=0)# Definiendo la función "Lr" con sus parámetros
    lr_lasso.fit(x_train_std,y_train) #Entrenando un algoritmo de Machine Learnig con el dataset x_train_std
    y_pred =lr_lasso.predict(x_test_std)
    from sklearn.metrics import f1_score
    return print('C= %f, penalty=L1,f1 test:%2f'% (c, sklearn.metrics.f1_score(y_test, y_pred)))

In [327]:
lr_lasso(0.01)
lr_lasso(0.1)
lr_lasso(1)

C= 0.010000, penalty=L1,f1 test:0.285714
C= 0.100000, penalty=L1,f1 test:0.362657
C= 1.000000, penalty=L1,f1 test:0.363958


## Comparación Coeficientes

In [329]:
def lr_ridge_coef(c):
    lr_ridge= LogisticRegression(solver='liblinear',C=c,penalty='l2',random_state=0)# Definiendo la función "Lr" con sus parámetros
    lr_ridge.fit(x_train_std,y_train) #Entrenando un algoritmo de Machine Learnig con el dataset x_train_std  
    return print(lr_ridge.coef_)

In [330]:
lr_ridge_coef(0.01)

[[ 0.1652191 -0.322609   0.0286169  0.2679218 -0.1687948  0.1516568
  -0.1658983 -0.2581105  0.1273893  0.2073783  0.7025499]]


In [331]:
lr_ridge_coef(0.1)

[[ 0.4365231 -0.5228868 -0.0388122  0.6267583 -0.2102074  0.2363162
  -0.2933516 -0.5773121  0.2914872  0.3084422  0.8289398]]


In [332]:
lr_ridge_coef(1)

[[ 0.5787133 -0.5500858 -0.057581   0.8131656 -0.1863789  0.252602
  -0.3331946 -0.8251086  0.37509    0.3464812  0.7747421]]


In [333]:
def lr_lasso_coef(c):
    lr_lasso= LogisticRegression(solver='liblinear',C=c,penalty='l1',random_state=0)# Definiendo la función "Lr" con sus parámetros
    lr_lasso.fit(x_train_std,y_train) #Entrenando un algoritmo de Machine Learnig con el dataset x_train_std  
    return print(lr_lasso.coef_)

In [334]:
lr_lasso_coef(0.01)

[[ 0.        -0.247048   0.         0.         0.         0.
   0.         0.         0.         0.015347   0.8012157]]


In [335]:
lr_lasso_coef(0.1)

[[ 0.2692553 -0.5301376  0.         0.4313487 -0.1981116  0.1959009
  -0.2184767 -0.2958936  0.1961613  0.2661656  0.9511966]]


In [336]:
lr_lasso_coef(1)

[[ 0.5677362 -0.5494197 -0.0533869  0.8018003 -0.1827472  0.2483202
  -0.3264602 -0.8094252  0.3690874  0.3435417  0.7817504]]


Para este caso el mejor f1 score es proporcionado por una regresión logística combinada con el algoritmo ridge con un valor de c=1, para los datos analizados los coeficientes son menores en todos los casos para las estimaciones realizadas bajo la metodología lasso.